In [145]:
# These are all the modules we'll be using later. 
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data I generated in build_a_libe_camera_1.ipynb.

In [233]:
pickle_file = 'SVHN.pickle'

with open(pickle_file, 'rb') as f:
  data = pickle.load(f)
  train_dataset = data['train_dataset']
  train_labels = data['train_labels']
  valid_dataset = data['valid_dataset']
  valid_labels = data['valid_labels']
  test_dataset = data['test_dataset']
  test_labels = data['test_labels']
  del data # To help gc free up memory
  print ('Training set', train_dataset.shape, train_labels.shape)
  print ('Validation set', valid_dataset.shape, valid_labels.shape)
  print ('Test set', test_dataset.shape, test_labels.shape)

Training set (228062, 32, 32, 1) (228062, 6)
Validation set (7693, 32, 32, 1) (7693, 6)
Test set (13068, 32, 32, 1) (13068, 6)


In [234]:
image_size = 32
num_labels = 6

def reformat(dataset):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  return dataset

train_dataset = reformat(train_dataset)
valid_dataset = reformat(valid_dataset)
test_dataset = reformat(test_dataset)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (228062, 1024) (228062, 6)
Validation set (7693, 1024) (7693, 6)
Test set (13068, 1024) (13068, 6)


In [235]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) 
          / predictions.shape[0])

In [270]:
batch_size = 64
image_size = 32
num_labels = 6
beta = 0.001

learning_rate = 0.5

num_units1 = 32*32*num_filters3
num_units2 = 1024

keep_prob = 0.5

beta = 0.001

In [278]:
graph = tf.Graph()
with graph.as_default():
    
  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_dataset_image = tf.reshape(tf_train_dataset, [-1,32,32,1])
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # conv & pool 1
  num_filters1 = 32
  W_conv1 = tf.Variable(tf.truncated_normal([5,5,1, num_filters1], stddev=0.1))
  h_conv1 = tf.nn.conv2d(tf_train_dataset_image, W_conv1, strides=[1,1,1,1], padding='SAME')
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
  h_conv1_cutoff = tf.nn.relu(h_conv1+ b_conv1)
  h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

  # conv & pool 2
  num_filters2 = 64
  W_conv2 = tf.Variable(tf.truncated_normal([5,5, num_filters1, num_filters2], stddev=0.1))
  h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
  h_conv2_cutoff = tf.nn.relu(h_conv2+ b_conv2)
  h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
  # conv & pool 3
  num_filters3 = 64
  W_conv3 = tf.Variable(tf.truncated_normal([5,5, num_filters2, num_filters3], stddev=0.1))
  h_conv3 = tf.nn.conv2d(h_pool2, W_conv3, strides=[1,1,1,1], padding='SAME')
  b_conv3 = tf.Variable(tf.constant(0.1, shape=[num_filters3]))
  h_conv3_cutoff = tf.nn.relu(h_conv3+ b_conv3)
  h_pool3 = tf.nn.max_pool(h_conv3_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

  # Dropout.
  h_pool3_flat = tf.reshape(h_pool3, [-1, 32*32*num_filters3])

  W = tf.Variable(tf.truncated_normal([num_units1, num_units2], stddev=0.1))
  b = tf.Variable(tf.constant(0.1, shape=[num_units2]))
  hidden = tf.nn.relu(tf.matmul(h_pool3_flat, W) + b)

  hidden_dropout = tf.nn.dropout(hidden, keep_prob)

  W0 = tf.Variable(tf.zeros([num_units2, num_labels]))
  b0 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  logits =  tf.matmul(hidden_dropout, W0) + b0
  loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels)) 

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
  # Predictions for the training, validation, and test data.
  train_h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)
  train_h2 = tf.nn.relu(tf.matmul(train_h1, weights_h2) + biases_h2)
  train_h3 = tf.nn.relu(tf.matmul(train_h2, weights_h3) + biases_h3)
  train_h4 = tf.nn.relu(tf.matmul(train_h3, weights_h4) + biases_h4)
  train_logits = tf.matmul(train_h4, weights_o) + biases_o
  train_prediction = tf.nn.softmax(train_logits)
  
  valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_h1)
  valid_h2 = tf.nn.relu(tf.matmul(valid_h1, weights_h2) + biases_h2)
  valid_h3 = tf.nn.relu(tf.matmul(valid_h2, weights_h3) + biases_h3)
  valid_h4 = tf.nn.relu(tf.matmul(valid_h3, weights_h4) + biases_h4)
  valid_logits = tf.matmul(valid_h4, weights_o) + biases_o
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_h1)
  test_h2 = tf.nn.relu(tf.matmul(test_h1, weights_h2) + biases_h2)
  test_h3 = tf.nn.relu(tf.matmul(test_h2, weights_h3) + biases_h3)
  test_h4 = tf.nn.relu(tf.matmul(test_h3, weights_h4) + biases_h4)
  test_logits = tf.matmul(test_h4, weights_o) + biases_o
  test_prediction = tf.nn.softmax(test_logits)


  

ValueError: ('Rank mismatch: Labels rank (received %s) should equal logits rank (received %s) - 1.', 2, 2)

In [268]:
num_steps=10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range(num_steps):
        offset=(step*batch_size)%(train_dataset.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            
        global_step+=1
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


initialized


ValueError: Fetch argument <tensorflow.python.framework.ops.Operation object at 0x12a61f710> of <tensorflow.python.framework.ops.Operation object at 0x12a61f710> cannot be interpreted as a Tensor. (Operation name: "Adam"
op: "AssignAdd"
input: "Variable"
input: "Adam/value"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@Variable"
    }
  }
}
attr {
  key: "use_locking"
  value {
    b: false
  }
}
 is not an element of this graph.)